In [73]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import unicodedata
from collections import Counter

[nltk_data] Downloading package punkt to C:\Users\Juan
[nltk_data]     Samuel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [74]:
data = pd.read_csv('processed_data.csv',encoding ='utf-8')
data.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,credit_card,...,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,boleto,...,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,credit_card,...,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,credit_card,...,45.00,27.20,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,credit_card,...,19.90,8.72,papelaria,38.0,316.0,4.0,250.0,51.0,15.0,15.0


In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95978 entries, 0 to 95977
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       95978 non-null  object 
 1   customer_id                    95978 non-null  object 
 2   order_status                   95978 non-null  object 
 3   order_purchase_timestamp       95978 non-null  object 
 4   order_approved_at              95978 non-null  object 
 5   order_delivered_carrier_date   95978 non-null  object 
 6   order_delivered_customer_date  95978 non-null  object 
 7   order_estimated_delivery_date  95978 non-null  object 
 8   payment_sequential             95978 non-null  int64  
 9   payment_type                   95978 non-null  object 
 10  payment_installments           95978 non-null  int64  
 11  payment_value                  95978 non-null  float64
 12  customer_unique_id             95978 non-null 

In [76]:
data = data.astype({"order_status" : 'category', \
                       "order_purchase_timestamp" : 'datetime64[ns]', "order_approved_at" : 'datetime64[ns]', \
                       "order_delivered_carrier_date" : 'datetime64[ns]', "order_delivered_customer_date" : 'datetime64[ns]', \
                       "order_estimated_delivery_date" : 'datetime64[ns]', 'review_creation_date':'datetime64[ns]', \
                        "review_answer_timestamp" : 'datetime64[ns]', "shipping_limit_date" : 'datetime64[ns]', \
                        "product_category_name" : 'category', 
                       })

Now, we want to analyse the review comment. We first create the stopwords. Stopwords are the words in any language which does not add much meaning to a sentence. We need to get rid of this stopwords in order for the sentiment analysis to be effective.

Source: https://medium.com/@saitejaponugoti/stop-words-in-nlp-5b248dadad47#:~:text=Stopwords%20are%20the%20words%20in,at%2C%20which%2C%20and%20on

In [77]:
stop_words = set(w for w in nltk.corpus.stopwords.words('portuguese'))

We will create few functions to help us do the sentiment analysis. First one is normalize. When we normalize text, we attempt to reduce its randomness, bringing it closer to a predefined “standard”. This helps us to reduce the amount of different information that the computer has to deal with, and therefore improves efficiency. Luckily, unicodata provide an easy normalize function. Next, we create the split_words function. This is a pre-step for the next function, we change all the sentence in tokens and we throw away the stopwords in the sentence. Tokenization is essentially splitting a phrase, sentence, paragraph, or an entire text document into smaller units, such as individual words or terms. Each of these smaller units are called tokens. Lastly, we create word to trigrams.A trigram is a three-word sequence of words like “please turn your”, or “turn your homework”. NLTK also provides the way to change the words into trigrams.

Hence, the purpose of these three functions is to change the sentence into trigrams that contain meaningful words.

Source:
https://www.analyticsvidhya.com/blog/2019/07/how-get-started-nlp-6-unique-ways-perform-tokenization/#:~:text=What%20is%20Tokenization%20in%20NLP%3F,-Tokenization%20is%20one&text=Tokenization%20is%20essentially%20splitting%20a,smaller%20units%20are%20called%20tokens



In [78]:
# Source
# https://stackoverflow.com/questions/51710082/what-does-unicodedata-normalize-do-in-python

def normalize(sentence):
    return unicodedata.normalize('NFKD',sentence).encode('ascii', errors='ignore').decode('utf-8')

def split_words(sentence):
    normalized = normalize(sentence.lower())
    tokens = nltk.tokenize.word_tokenize(normalized) #creating tokens
    words = tuple(t for t in tokens if t not in stop_words and t.isalpha()) #remove stopwords
    return words

def words_to_trigrams(words):
    collection = []
    for word in words:
        collection.extend(' '.join(trigram) for trigram in nltk.trigrams(word))
    return collection

Then, we use the function above to split into words. We then separate the data into positive and negative reviews. Positive reviews have the review score 4 or 5 while negative reviews have the review score 1 or 2. Lastly, we change it into trigrams.

In [88]:
data['review_comment_words'] = data['review_comment_message'].apply(split_words)

positive_trigrams = words_to_trigrams( data[(data['review_score'] == 4) | (data['review_score']==5)].review_comment_words)
negative_trigrams = words_to_trigrams( data[(data['review_score'] == 1) | (data['review_score']==2)].review_comment_words)

From the trigrams, we select 15 of the most common trigrams from both positive and negative. The module collection in Python is one of the most useful module. We will use Counter object and use its method most common to count and select the 15 most common trigrams. Last but not least, we join the positive trigrams and negative trigrams into one dataframe.

Source: https://docs.python.org/3/library/collections.html

In [93]:
top_positive = dict(Counter(positive_trigrams).most_common(15))
top_negative = dict(Counter(negative_trigrams).most_common(15))


review_trigrams = pd.DataFrame(list(zip(top20_positive.keys(), top20_positive.values(), \
                                top20_negative.keys(),top20_negative.values())), \
                                columns =['positive','count', 'negative', 'count'])

In [94]:
review_trigrams

,positive,count,negative,count
0,chegou antes prazo,1047,nao recebi produto,715
1,bem antes prazo,669,ainda nao recebi,368
2,entregue antes prazo,585,ate agora nao,195
3,produto chegou antes,412,produto nao entregue,192
4,entrega antes prazo,406,ate momento nao,133
5,chegou bem antes,402,produto ainda nao,126
6,produto entregue antes,361,produto nao chegou,120
7,entrega super rapida,321,ainda nao chegou,104
8,produto otima qualidade,282,momento nao recebi,93
9,antes prazo previsto,266,ate agora nada,87


try to change to english?

In [95]:
from google_trans_new import google_translator  
  
translator = google_translator()  

for i in range(review_trigrams.shape[0]) :
    review_trigrams.iloc[i,0] = translator.translate(review_trigrams.iloc[i,0], lang_tgt='en')
    review_trigrams.iloc[i,2] = translator.translate(review_trigrams.iloc[i,2], lang_tgt='en')
    
    
review_trigrams

,positive,count,negative,count
0,arrived before deadline,1047,I did not recieve the product,715
1,well-term,669,I still have not received,368
2,delivered before time,585,So far no,195
3,product has come before,412,product not delivered,192
4,Delivery before time,406,Until now,133
5,arrived well before,402,product not yet,126
6,product delivered before,361,product has not arrived,120
7,Super fast delivery,321,it did not arrive yet,104
8,product otima quality,282,moment I did not receive,93
9,before expected,266,"until now, nothing",87


As we can see from the trigrams, we can see the positive trigram is mostly connected to the delivery time, such as "arrived before deadline" or "Super fast delivery".Even more, some of the negative trigram is also connected to the delivery time, e.g. "product has not arrived" or "it did not arrive yet". From this trigram, it drives us to a hypotheses that the review score is highly connected to the delivery time.